# debug datarmor

tests are conducted with zarr first and then with netcdf on the rotary spectrum computation

In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

import datetime
now = datetime.datetime.now

from mitequinox.utils import *

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster()
w = cluster.scale(20)

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:134: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [7]:
client

Client Scheduler: tcp://10.148.0.39:41906 Dashboard: http://10.148.0.39:8787/status,Cluster Workers: 10 Cores: 40 Memory: 250.00 GB


In [5]:
# main output directory
scratchd = '/home/c11-data/Test_aponte/'

In [6]:
# zarr compression
# http://xarray.pydata.org/en/stable/io.html
# http://zarr.readthedocs.io/en/stable/tutorial.html#compressors
compressor = None

---

# store grid

In [8]:
ds_index, ds = get_compressed_level_index(grid_dir)

# we will need to keep some of these when computing gradients
#ds = ds.drop(['dxG','dyG','dxC','dyC','rAw','rAs','rAz'])
ds = ds.drop(['hFacC','hFacW','hFacS'])
ds = ds.drop(['Z', 'Zp1', 'Zu', 'Zl', 'drC', 'drF','PHrefC','PHrefF'])

Nc = 432 # original choice
#Nc = 27 # very long scheduling
#Nc = 96 # 96*45
ds = ds.chunk({'i': Nc, 'j': Nc, 'i_g': Nc, 'j_g': Nc})

#print(ds)

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


In [9]:
out_dir = scratchd+'mit/'

In [10]:
file_out = out_dir+'grid.zarr'
print(now())
%time ds.to_zarr(file_out, mode='w')

2018-10-17 09:35:51.671038
CPU times: user 23.1 s, sys: 956 ms, total: 24.1 s
Wall time: 31.5 s


--- 

# zarr case

In [8]:
V = ['SSU', 'SSV']

# faces
F = range(13)
F = range(2)

# parameters for rechunking
Nt = 24*10 # time chunks
#
#Nc = 96 # x 45 = 4320
#Nc = 48 # x 90 = 4320
#Nc = (96, 48)
# other choices: 432, 27 (long scheduling), 288, 864
Nc = (120, 60) # for dij slices

In [9]:
# try to reintore compression once the computation goes through
# try to delete del statements

#overwrite = False
overwrite = True

for face in F:

    Efile = scratchd+'mit/spectra/%s_f%02d.zarr'%('SSUV',face)
    
    if not os.path.isdir(Efile) or overwrite:
    
        if True:
            # binary to zarr (standard layout)
            print('++ zarr conversion, standard layout: ', now())
            %time zarr_standard(V, client,  F=face, out_dir=scratchd+'mit/standard/')

        # rechunk all variables
        print('++ zarr rechunking and storage: ', now())
        %time rechunk(V, F=face, in_dir=scratchd+'mit/standard/', out_dir=scratchd+'mit/rechunked/', \
                      Nt=Nt, Nc=Nc, dij=4)

        # load data
        dsu = xr.open_zarr(scratchd+'mit/rechunked/%s_f%02d.zarr'%('SSU',face))
        dsv = xr.open_zarr(scratchd+'mit/rechunked/%s_f%02d.zarr'%('SSV',face))
        ds = xr.merge([dsu,dsv], compat='identical')

        # try to delete this file writing once the computation goes through
        #ds['SSU'] = ds['SSU'].where(abs(ds['SSU'])<10) # tmp: SSU.0000659952.shrunk is corrupted
        ds['SSUV'] = ds['SSU'].rename({'i_g': 'i'}).fillna(0.)  \
                     + 1j * ds['SSV'].rename({'j_g': 'j'}).fillna(0.)
        #del ds['SSU'], ds['SSV']
        #store_zarr(ds, scratch+'mit/spectra/SSUV_tmp.zarr', 
        #           encoding={'SSUV': {'compressor': None}})
        #print('SSUV stored')
        #del ds
        #ds = xr.open_zarr(scratch+'mit/spectra/SSUV_tmp.zarr')

        # need to run once to get frequency
        f, E = get_E(ds['SSUV'].isel(i=slice(2),j=slice(2)))
        E = get_E(ds['SSUV'], f=f)

        # store
        E = E.to_dataset()
        # tmp, waiting for xarray release 
        for v in E.variables:
            if hasattr(E[v],'encoding') and 'chunks' in E[v].encoding:
                del E[v].encoding['chunks']    
        #
        print('++ spectral calculation and storage: ', now())
        %time E.to_zarr(Efile, mode='w', \
                        encoding={'SSUV': {'compressor': None}})
        del E, ds

        # clean up
        #for v in V:
        #    shutil.rmtree(scratchd+'mit/standard/%s.zarr' %v)
        #    shutil.rmtree(scratchd+'mit/rechunked/%s_f%02d.zarr'%(v,face))
    
    print('--- face %d done'%face)

--- face 0 done
++ zarr conversion, standard layout:  2018-10-17 09:46:35.388453


/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


OSError: [Errno 122] Disk quota exceeded: '/home/c11-data/Test_aponte/mit/standard/SSU.zarr/SSU/4721.0.0.0._lr2qw4x.partial'

++ zarr rechunking and storage:  2018-10-17 09:47:30.626363


OSError: [Errno 122] Disk quota exceeded: '/home/c11-data/Test_aponte/mit/rechunked/SSU_f01.zarr/SSU/18.6.1.qseo5h1y.partial'

ValueError: group not found at path ''

---

In [17]:
w = cluster.start_workers(10)

In [ ]:
client.restart()

In [10]:
# kill scheduler, workers
cluster.scheduler.close()
#cluster.stop_workers(cluster.jobs)

<Future pending cb=[_make_coroutine_wrapper.<locals>.wrapper.<locals>.<lambda>() at /home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/tornado/gen.py:347]>